In [1]:
import numpy as np
import numpy.linalg as la
# import scipy.sparse as sparse

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.patches import Circle


import timeit
import numba
from numba import jit, njit

from iteration_methods import*
from basic import*

In [2]:
#output settings
np.set_printoptions(precision=2)
%matplotlib qt 

In [3]:
@njit
def source(r, th, u):

    f = 1/np.sin(th)

    # f = sigma*s**2*(u-uc)**(2*sigma-1)

    return f

@njit
def source_1(r, th, u, params = (1, 1, 0)):

    # f1 = r**2/u

    sigma, s, uc = params
        
    f1 = sigma*s**2*(u-uc)**(2*sigma-1)

    # if (u-uc) <= 0.:

    #     f1=f1*0.
    
    return f1

@njit
def u_analytical(r, th):

    uan = r**2*np.sin(th) 

    return uan

In [4]:
def grad_shaf_solver(matrix, source_term, grid, init_guess=None, 
                    boundary=((0, 0), (0,0)), tolerance=1.e-8, 
                    itermax=1000, omega=1.5, params=(1, 1, 0)):

    #read the input
    M, u0, B, tol, kmax = matrix, init_guess, boundary, tolerance, itermax

    #assign the grid
    r, th = grid

    R, TH = r[0], th[:,0]

    Nr, Nth = r.shape[1]-1, th.shape[0]-1

    dr, dth = (r[0, -1] - r[0, 0])/Nr, (th[-1, 0] - th[0, 0])/Nth

    
    #assign the initial guess
    if init_guess is None:

        u0 = np.ones_like(r)

    #assign dirichlet boundary conditions
    u0[1:-1, 0] = B[0][0][1:-1]                               
    # u0[:, -1] = B[0][1]
    u0[0, :] = B[1][0]
    u0[-1, :] = B[1][1]
    
    #assign extra parameters
    sigma, s, uc = params

    #assign the source term
    f = source_term(r, th, u0)#*np.heaviside(u0-uc, 0)
    
    #initial values before the iteration loop starts
    u = u0.copy()
    k = 0
    rel_diff = tol + 1
    conv_hist = []

    

    #iteration loop 
    while  k < kmax and rel_diff > tol:    
        
        #print the iteration number to keep track of the solver
        # if np.mod(k, 100) == 0:

            # print(k)

        u_next = u.copy()

        #Update the source term if it is a function of the solution
        f = source_term(r, th, u)#*np.heaviside(u-uc, 0)

        #calculate the solution in the kth step
        for j in range(1, Nth):
            for i in range(1, Nr+1):
                
                #simulate the effect of the heaviside step function
                if u[j,i] < uc:

                    f[j, i] *= 0

                #Robin boundary conditions at the outermost radius
                if i == Nr:

                    u_next[j,i] = (1-omega)*u[j,i] + omega/((2+4*dr/R[i])*R[i]**2*dth**2 + 2*dr**2)*(R[i]**2*dth**2*(2*u_next[j, i-1])                                                                                        + dr**2*(u[j+1,i]*(1+dth/(2*np.tan(TH[j]))) + 
                                                                                        u_next[j-1,i]*(1 - dth/(2*np.tan(TH[j])))) 
                                                                                        -f[j,i]*dr**2*dth**2*R[i]**2)
                    
                else:

                    u_next[j,i] = (1-omega)*u[j,i] + omega/(2*(R[i]**2*dth**2 + dr**2))*(R[i]**2*dth**2*(u[j,i+1] + u_next[j, i-1]) + 
                                                                                        dr**2*(u[j+1,i]*(1+dth/(2*np.tan(TH[j]))) + 
                                                                                        u_next[j-1,i]*(1 - dth/(2*np.tan(TH[j])))) 
                                                                                        -f[j,i]*dr**2*dth**2*R[i]**2)
                
        #calculate the L2 norm of the relative difference between the two last iterations   
        rel_diff = la.norm(u_next-u)/la.norm(u)
        
        #Save the convergence history
        conv_hist.append(rel_diff)

        #update solution for next iteration
        u = u_next

        k += 1

    return u, k, rel_diff, conv_hist

In [5]:
#Setup the parameters for the solver. Function grid() is in the basic.py module

#The grid of the problem
R, TH, r, th, dr, dth = polar_grid(th_I=0, th_F=np.pi, r_I=0.1, r_F=1., Nr=100, Nth=100)            

#initial guess
u_guess = np.sin(th)**2/r       

#extra parameters that may be needed
sigma = 1.1
s = 1.6
uc = 0.5
params = (sigma , s, uc)

#source term
f = source_1(r, th, u_guess, params)      

#analytical solution for comparison
u_an = u_analytical(r, th)

#boundary conditions
boundary = ((np.sin(TH)**2, 0), (0., 0.))                                   

#max number of iteration
iterations = 10000                              

#desired tolerance
tolerance = 1.e-8                               

#relaxation parameter for SOR method
omega_opt = 2/(1+np.sin(np.pi*max(dr, dth)))     

In [6]:
#jit the solver function
grad_shaf_solver = jit(nopython=True)(grad_shaf_solver)

In [7]:
#Calculate, time and qualify the solution

start = timeit.default_timer()

u, k, rel_diff, conv_hist = grad_shaf_solver(np.eye(2), source_1, (r, th), init_guess=u_guess, boundary=boundary,                                                                         tolerance=tolerance, itermax=iterations, omega=omega_opt, params=params)

# u1, k1, rel_diff1, conv_hist1 = grad_shaf_solver(np.eye(99), source_1, (r, th), init_guess=u_guess, boundary=boundary,                                                                         tolerance=tolerance, itermax=iterations, omega=omega_opt, params=params)

elapsed = timeit.default_timer() - start

#Relative error to analytical
error_to_an = la.norm(u - u_an, 2)/la.norm(u_an, 2)

#print solution
print('SOR solver \nNumber of iterations: {:} \nLast Relative difference: {:1.3E}' 
      '\nError to analytical: {:1.3E}s \nElapsed time: {:1.2E}s '.format(k, rel_diff, error_to_an, elapsed))
print('-'*50)

SOR solver 
Number of iterations: 2805 
Last Relative difference: 9.969E-09
Error to analytical: 9.976E-01s 
Elapsed time: 1.98E+01s 
--------------------------------------------------


In [8]:
#Plot results. Function plot() is in basic.py module.
from basic import plot

plot(r*np.cos(th), r*np.sin(th), u, u1=None, u_an=None, conv_hist=conv_hist, plot_result=True)

In [9]:
# #Creat 3D figure
# fig = plt.figure()
# ax = fig.add_subplot(111, projection = '3d')

# #Scatter plot of  u
# surf = ax.plot_surface(r*np.cos(th), r*np.sin(th), u, cmap = cm.coolwarm, label = 'numerical', alpha = 0.5)

# ax.set_xlabel('x')
# ax.set_ylabel('y')
# ax.set_title('Magnetar magnetic field')

# surf._facecolors2d=surf._facecolors3d
# surf._edgecolors2d=surf._edgecolors3d
# ax.legend(loc = 2)


# # ax.contour3D(r*np.cos(th), r*np.sin(th), u)

# cset = ax.contour(r*np.cos(th), r*np.sin(th), u, 30, zdir='z', offset=0, cmap=cm.coolwarm)

# plt.show()


In [11]:
fig2, ax2 = plt.subplots()

# ax2.plot(x, y)
# img = ax2.imshow(u, origin='lower', cmap='YlGnBu_r', extent=[0, 1, -0.5, 0.5])
cont = ax2.contour(r*np.sin(th), r*np.cos(th), u, levels=20, extent=[-0.5, 0.5, 0, 1])

# cbar_ax = make_axes_locatable(ax2).append_axes(position='right', size='5%', pad=0.1)

# cbar = fig2.colorbar(mappable=img, cax=cbar_ax)

# ax2.contour(r*np.sin(th), r*np.cos(th), np.sin(th)**2/r, levels=20, extent=[-0.5, 0.5, 0, 1], alpha=0.5)

ax2.add_patch(Circle((0.,0.), 0.1, color='b', zorder=100))

ax2.set_ylim(top=R[-1]*np.cos(TH[0])/2, bottom=R[-1]*np.cos(TH[-1])/2)

# ax2.axis('scaled')

(-0.5, 0.5)

NameError: name 'matplotlib' is not defined